This notebook is for getting emoji descriptions from site https://emojipedia.org

In [68]:
%load_ext autoreload
%autoreload 2

import urllib
from bs4 import *
import re
from utils import *
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
LINK_TO_EMOJI_REGEXP = re.compile("<a href=\"(.+)\"><span class=\"emoji\">")
TITLE_REGEXP = re.compile("<title>(.+)Emoji</title>")
EMOJI_DEF_REGEXP = re.compile("</span>(.+)</li>")

cite = 

In [29]:
def get_links_to_emoji(html_page):
    soup = BeautifulSoup(html_page, 'html.parser')
    links_to_emoji = []
    for lines in soup:
        lines = str(lines).split('\n')
        for i, line in enumerate(lines):
            line = str(line)
            if line == "<ul class=\"emoji-list\">":
                while lines[i+1] != "</ul>":
                    i += 1
                    line = lines[i]
                    link_to_emoji = re.findall(LINK_TO_EMOJI_REGEXP, line)
                    links_to_emoji.append(link_to_emoji[0])
                break
    return links_to_emoji

In [100]:
def get_emoji_name_and_defs(html_page):
    soup = BeautifulSoup(html_page, 'html.parser')
    title_tag = soup("title")
    try:
        title = re.findall(TITLE_REGEXP, str(title_tag))[0]
    except Exception as e:
        print(str(title_tag))
    title = title.split()
    smile = title[0].strip()
    smile_name = ' '.join([x.strip() for x in title[1:]])
    
    emoji_defs = [smile_name]
    for lines in soup:
        lines = str(lines).split('\n')
        for i, line in enumerate(lines):
            line = str(line)
            if line == "<h2>Also Known As</h2>":
                i += 1
                while lines[i+1] != "</ul>":
                    i += 1
                    line = lines[i]
                    emoji_def = re.findall(EMOJI_DEF_REGEXP, line)[0]
                    emoji_defs.append(emoji_def.strip())
                break
    return smile, smile_name, emoji_defs

In [147]:
emoji_categories = ['/people/',
 '/nature/',
 '/food-drink/',
 '/activity/',
 '/travel-places/',
 '/objects/',
 '/symbols/',
 '/flags/' ]

In [148]:
emojis = []

In [149]:
import time
for category in emoji_categories:
    print("Starting category:", category[1:-1])
    category_link = "https://emojipedia.org{}".format(category)
    category_html_page = urllib.request.urlopen(category_link)
    links_to_emoji = get_links_to_emoji(category_html_page)
    for i, link_to_emoji in enumerate(links_to_emoji):
        try:
            emoji_link = "https://emojipedia.org{}{}".format(category, link_to_emoji)
            emoji_html_page = urllib.request.urlopen(emoji_link)
        except Exception as e:
            emoji_link = "https://emojipedia.org{}".format(link_to_emoji)
            emoji_html_page = urllib.request.urlopen(emoji_link)
        data = get_emoji_name_and_defs(emoji_html_page)
        emojis.append(data)
        print(data[0], data[2])
        time.sleep(0.1)

Starting category: flags
🏁 ['Chequered Flag', 'Checkered Flag', 'Grid Girl', 'Racing Flag']
🚩 ['Triangular Flag on Post', 'Flag on Pole', 'Red Flag']
🎌 ['Crossed Flags', 'Two Flags']
🏴 ['Waving Black Flag']
🏳️ ['Waving White Flag']
🏳️‍🌈 ['Rainbow Flag', 'Pride Flag']
🇦🇨 ['Flag for Ascension Island']
🇦🇩 ['Flag for Andorra', 'Andorran Flag']
🇦🇪 ['Flag for United Arab Emirates', 'Emirati Flag', 'UAE Flag']
🇦🇫 ['Flag for Afghanistan', 'Afghan Flag']
🇦🇬 ['Flag for Antigua &amp; Barbuda']
🇦🇮 ['Flag for Anguilla', 'Anguillan Flag']
🇦🇱 ['Flag for Albania', 'Albanian Flag']
🇦🇲 ['Flag for Armenia', 'Armenian Flag']
🇦🇴 ['Flag for Angola', 'Angolan Flag']
🇦🇶 ['Flag for Antarctica', 'Antarctic Flag']
🇦🇷 ['Flag for Argentina', 'Argentinian Flag']
🇦🇸 ['Flag for American Samoa', 'American Samoan Flag']
🇦🇹 ['Flag for Austria', 'Austrian Flag']
🇦🇺 ['Flag for Australia', 'Aussie Flag', 'Australian Flag']
🇦🇼 ['Flag for Aruba', 'Aruban Flag']
🇦🇽 ['Flag for Åland Islands', 'Åland Flag']
🇦🇿 ['Flag for Azerba

🇲🇽 ['Flag for Mexico', 'Mexican Flag']
🇲🇾 ['Flag for Malaysia', 'Malaysian Flag']
🇲🇿 ['Flag for Mozambique', 'Mozambican Flag']
🇳🇦 ['Flag for Namibia', 'Namibian Flag']
🇳🇨 ['Flag for New Caledonia', 'New Caledonian Flag']
🇳🇪 ['Flag for Niger', 'Nigerien\xa0Flag']
🇳🇫 ['Flag for Norfolk Island', 'Norfolk Island Flag']
🇳🇬 ['Flag for Nigeria', 'Nigerian Flag']
🇳🇮 ['Flag for Nicaragua', 'Nicaraguan Flag']
🇳🇱 ['Flag for Netherlands', 'Dutch Flag']
🇳🇴 ['Flag for Norway', 'Norwegian Flag']
🇳🇵 ['Flag for Nepal', 'Nepalese Flag']
🇳🇷 ['Flag for Nauru', 'Nauruan Flag']
🇳🇺 ['Flag for Niue', 'Niuean Flag']
🇳🇿 ['Flag for New Zealand', 'Kiwi Flag']
🇴🇲 ['Flag for Oman', 'Omani Flag']
🇵🇦 ['Flag for Panama', 'Panamanian Flag']
🇵🇪 ['Flag for Peru', 'Peruvian Flag']
🇵🇫 ['Flag for French Polynesia', 'French Polynesian Flag']
🇵🇬 ['Flag for Papua New Guinea', 'Papua New Guinean Flag', 'PNG Flag']
🇵🇭 ['Flag for Philippines']
🇵🇰 ['Flag for Pakistan', 'Pakistani Flag']
🇵🇱 ['Flag for Poland', 'Polish Flag']
🇵🇲 ['

In [153]:
len(emojis)

1625

In [162]:
with open("emojipedia_data.txt", "w") as file:
    for i in range(len(emojis)):
        for j in range(3):
            file.write(str(emojis[i][j]))
            file.write('\t')
        file.write('\n')